# CYBERML Project

In [1]:
import pandas as pd
import numpy as np

DATA_FOLDER = './data/'
ATTACK_FOLDER = 'attack_data/'
BENIGN_FOLDER = 'benign_data/'

## Load data

In [2]:
attack_data = pd.read_csv(DATA_FOLDER + ATTACK_FOLDER + 'attack_samples_1sec.csv')
attack_data.head()

,device_name,device_mac,label_full,label1,label2,label3,label4,timestamp,timestamp_start,timestamp_end,...,network_time-delta_min,network_time-delta_std_deviation,network_ttl_avg,network_ttl_max,network_ttl_min,network_ttl_std_deviation,network_window-size_avg,network_window-size_max,network_window-size_min,network_window-size_std_deviation
0,edge1,dc:a6:32:dc:27:d4,attack_ddos_syn-flood-port-80_edge1,attack,ddos,syn-flood-port-80,ddos_syn-flood-port-80,2025-01-23T15:31:10.709000Z_2025-01-23T15:31:1...,2025-01-23T15:31:10.709000Z,2025-01-23T15:31:11.709000Z,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,edge1,dc:a6:32:dc:27:d4,attack_ddos_syn-flood-port-80_edge1,attack,ddos,syn-flood-port-80,ddos_syn-flood-port-80,2025-01-23T15:31:11.709000Z_2025-01-23T15:31:1...,2025-01-23T15:31:11.709000Z,2025-01-23T15:31:12.709000Z,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,edge1,dc:a6:32:dc:27:d4,attack_ddos_syn-flood-port-80_edge1,attack,ddos,syn-flood-port-80,ddos_syn-flood-port-80,2025-01-23T15:31:12.709000Z_2025-01-23T15:31:1...,2025-01-23T15:31:12.709000Z,2025-01-23T15:31:13.709000Z,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,edge1,dc:a6:32:dc:27:d4,attack_ddos_syn-flood-port-80_edge1,attack,ddos,syn-flood-port-80,ddos_syn-flood-port-80,2025-01-23T15:31:13.709000Z_2025-01-23T15:31:1...,2025-01-23T15:31:13.709000Z,2025-01-23T15:31:14.709000Z,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,edge1,dc:a6:32:dc:27:d4,attack_ddos_syn-flood-port-80_edge1,attack,ddos,syn-flood-port-80,ddos_syn-flood-port-80,2025-01-23T15:31:14.709000Z_2025-01-23T15:31:1...,2025-01-23T15:31:14.709000Z,2025-01-23T15:31:15.709000Z,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
benign_data = pd.read_csv(DATA_FOLDER + BENIGN_FOLDER + 'benign_samples_1sec.csv')
benign_data.head()

,device_name,device_mac,label_full,label1,label2,label3,label4,timestamp,timestamp_start,timestamp_end,...,network_time-delta_min,network_time-delta_std_deviation,network_ttl_avg,network_ttl_max,network_ttl_min,network_ttl_std_deviation,network_window-size_avg,network_window-size_max,network_window-size_min,network_window-size_std_deviation
0,router,28:87:ba:bd:c6:6c,benign_whole-network3,benign,benign,benign,benign,2025-09-09T14:09:40.400000Z_2025-09-09T14:09:4...,2025-09-09T14:09:40.400000Z,2025-09-09T14:09:41.400000Z,...,0.0,0.006059,62.800000,64.0,61.0,1.469694,1870.500000,3081.0,660.0,1210.500000
1,router,28:87:ba:bd:c6:6c,benign_whole-network3,benign,benign,benign,benign,2025-09-09T14:09:41.400000Z_2025-09-09T14:09:4...,2025-09-09T14:09:41.400000Z,2025-09-09T14:09:42.400000Z,...,0.0,0.016469,62.500000,64.0,61.0,1.500000,1870.500000,3081.0,660.0,1210.500000
2,router,28:87:ba:bd:c6:6c,benign_whole-network3,benign,benign,benign,benign,2025-09-09T14:09:42.400000Z_2025-09-09T14:09:4...,2025-09-09T14:09:42.400000Z,2025-09-09T14:09:43.400000Z,...,0.0,0.034312,61.571429,64.0,53.0,3.736199,2441.285714,4736.0,135.0,1813.237335
3,router,28:87:ba:bd:c6:6c,benign_whole-network3,benign,benign,benign,benign,2025-09-09T14:09:43.400000Z_2025-09-09T14:09:4...,2025-09-09T14:09:43.400000Z,2025-09-09T14:09:44.400000Z,...,0.0,0.012790,62.500000,64.0,61.0,1.500000,1870.500000,3081.0,660.0,1210.500000
4,router,28:87:ba:bd:c6:6c,benign_whole-network3,benign,benign,benign,benign,2025-09-09T14:09:44.400000Z_2025-09-09T14:09:4...,2025-09-09T14:09:44.400000Z,2025-09-09T14:09:45.400000Z,...,0.0,0.017764,62.800000,64.0,61.0,1.469694,2112.600000,3081.0,660.0,1186.042933


In [4]:
print("Attack Data Shape:", attack_data.shape)
print("Benign Data Shape:", benign_data.shape)

Attack Data Shape: (90391, 94)
Benign Data Shape: (136800, 94)


In [5]:
# Merge both datasets for training
combined_data = pd.concat([attack_data, benign_data], ignore_index=True)
combined_data = combined_data.sample(frac=1).reset_index(drop=True)  # Shuffle the data

print("Combined Data Shape:", combined_data.shape)
combined_data.head()

Combined Data Shape: (227191, 94)


,device_name,device_mac,label_full,label1,label2,label3,label4,timestamp,timestamp_start,timestamp_end,...,network_time-delta_min,network_time-delta_std_deviation,network_ttl_avg,network_ttl_max,network_ttl_min,network_ttl_std_deviation,network_window-size_avg,network_window-size_max,network_window-size_min,network_window-size_std_deviation
0,light-sensor,8c:aa:b5:8a:a9:b4,benign_whole-network3,benign,benign,benign,benign,2025-09-09T14:16:08.400000Z_2025-09-09T14:16:0...,2025-09-09T14:16:08.400000Z,2025-09-09T14:16:09.400000Z,...,0.000000e+00,0.001012,159.50,255.0,64.0,95.500000,34847.0,64056.0,5638.0,29209.0
1,flame-sensor,08:b6:1f:82:ee:cc,attack_recon_vuln-scan_flame-sensor,attack,recon,vuln-scan,recon_vuln-scan,2025-01-15T19:12:59.152000Z_2025-01-15T19:13:0...,2025-01-15T19:12:59.152000Z,2025-01-15T19:13:00.152000Z,...,3.400000e-08,0.013483,150.98,255.0,37.0,104.109268,3384.0,5744.0,1024.0,2360.0
2,rfid-sensor,08:b6:1f:82:2b:1c,attack_mitm_ip-spoofing_mqtt-broker--rfid-sensor,attack,mitm,ip-spoofing,mitm_ip-spoofing,2025-02-13T22:30:41.612000Z_2025-02-13T22:30:4...,2025-02-13T22:30:41.612000Z,2025-02-13T22:30:42.612000Z,...,2.700000e-08,0.000060,64.00,64.0,64.0,0.000000,0.0,0.0,0.0,0.0
3,plug-proximity,d4:a6:51:20:0e:3f,benign_whole-network3,benign,benign,benign,benign,2025-09-09T14:25:30.400000Z_2025-09-09T14:25:3...,2025-09-09T14:25:30.400000Z,2025-09-09T14:25:31.400000Z,...,5.615300e-02,0.000000,255.00,255.0,255.0,0.000000,0.0,0.0,0.0,0.0
4,water-sensor,08:b6:1f:84:66:78,benign_whole-network3,benign,benign,benign,benign,2025-09-09T14:13:33.400000Z_2025-09-09T14:13:3...,2025-09-09T14:13:33.400000Z,2025-09-09T14:13:34.400000Z,...,6.010000e-04,0.008700,159.50,255.0,64.0,95.500000,34857.0,64070.0,5644.0,29213.0


## Try learning

### Prepare data

In [6]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, confusion_matrix

# Replace all NaN values with zeros
combined_data.fillna(0, inplace=True)

# Copy data
encoded_data = combined_data.copy()

encoded_values = {}

# Turn label data into integers
label_columns = ['label1', 'label2', 'label3', 'label4']
for col in label_columns:
    encoded_data[col], uniques = pd.factorize(encoded_data[col])
    encoded_values[col] = dict(enumerate(uniques))

# Remove non numerical columns
encoded_data = encoded_data.select_dtypes(include=[np.number])
encoded_data.head()

encoded_data

,label1,label2,label3,label4,log_data-ranges_avg,log_data-ranges_max,log_data-ranges_min,log_data-ranges_std_deviation,log_data-types_count,log_interval-messages,...,network_time-delta_min,network_time-delta_std_deviation,network_ttl_avg,network_ttl_max,network_ttl_min,network_ttl_std_deviation,network_window-size_avg,network_window-size_max,network_window-size_min,network_window-size_std_deviation
0,0,0,0,0,915.0,915.0,915.0,0.0,2,0.0,...,0.000000e+00,0.001012,159.500000,255.0,64.0,95.500000,34847.00000,64056.0,5638.0,29209.000000
1,1,1,1,1,0.0,0.0,0.0,0.0,0,0.0,...,3.400000e-08,0.013483,150.980000,255.0,37.0,104.109268,3384.00000,5744.0,1024.0,2360.000000
2,1,2,2,2,0.0,0.0,0.0,0.0,0,0.0,...,2.700000e-08,0.000060,64.000000,64.0,64.0,0.000000,0.00000,0.0,0.0,0.000000
3,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,...,5.615300e-02,0.000000,255.000000,255.0,255.0,0.000000,0.00000,0.0,0.0,0.000000
4,0,0,0,0,1023.0,1023.0,1023.0,0.0,1,0.0,...,6.010000e-04,0.008700,159.500000,255.0,64.0,95.500000,34857.00000,64070.0,5644.0,29213.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227186,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.000000e+00,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000
227187,1,4,31,34,0.0,0.0,0.0,0.0,0,0.0,...,3.300000e-08,0.002862,140.370732,255.0,61.0,93.594867,34300.15122,65535.0,0.0,29876.242527
227188,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.000000e+00,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.000000
227189,0,0,0,0,137.0,137.0,137.0,0.0,1,0.0,...,1.000000e-06,0.002661,159.500000,255.0,64.0,95.500000,34854.50000,64061.0,5648.0,29206.500000


In [7]:
X = encoded_data.drop(['label1', 'label2', 'label3', 'label4'], axis=1)
y = encoded_data['label1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Supervised Learning

### Train With Gradient Boosting

In [8]:
import xgboost

modelGboost = xgboost.XGBClassifier()

modelGboost.fit(X_train, y_train)

y_pred = modelGboost.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[27226    56]
 [ 2355 15802]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     27282
           1       1.00      0.87      0.93     18157

    accuracy                           0.95     45439
   macro avg       0.96      0.93      0.94     45439
weighted avg       0.95      0.95      0.95     45439



### Train with Logistic regression

In [9]:
modelLinear = sklearn.linear_model.LogisticRegression()
modelLinear.fit(X_train, y_train)
y_pred = modelLinear.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[27145   137]
 [ 4876 13281]]
              precision    recall  f1-score   support

           0       0.85      0.99      0.92     27282
           1       0.99      0.73      0.84     18157

    accuracy                           0.89     45439
   macro avg       0.92      0.86      0.88     45439
weighted avg       0.90      0.89      0.89     45439



/home/baptiste/cyberml/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Train with SVC

In [10]:
model = LinearSVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[27018   264]
 [ 4336 13821]]
              precision    recall  f1-score   support

           0       0.86      0.99      0.92     27282
           1       0.98      0.76      0.86     18157

    accuracy                           0.90     45439
   macro avg       0.92      0.88      0.89     45439
weighted avg       0.91      0.90      0.90     45439



## Unsupervised learning

### Isolation Forest

In [11]:
model = sklearn.ensemble.IsolationForest(contamination=0.39)
model.fit(X_train)

y_pred = model.predict(X_test)

y_pred = (y_pred == (-1 if encoded_values["label1"][1] == "attack" else 1)).astype(int)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[21411  5871]
 [ 6315 11842]]
              precision    recall  f1-score   support

           0       0.77      0.78      0.78     27282
           1       0.67      0.65      0.66     18157

    accuracy                           0.73     45439
   macro avg       0.72      0.72      0.72     45439
weighted avg       0.73      0.73      0.73     45439



### SVM

In [20]:
X_train_sampled = X_train.sample(frac=0.1)
model = sklearn.svm.OneClassSVM(kernel="linear", shrinking=False)
model.fit(X_train_sampled)

y_pred = model.predict(X_test)

y_pred = (y_pred == 1).astype(int)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[18715  8567]
 [ 3936 14221]]
              precision    recall  f1-score   support

           0       0.83      0.69      0.75     27282
           1       0.62      0.78      0.69     18157

    accuracy                           0.72     45439
   macro avg       0.73      0.73      0.72     45439
weighted avg       0.75      0.72      0.73     45439



### Local Outlier Factor

In [21]:
import sklearn.neighbors

X_train_sampled_2 = X_train.sample(frac=0.5)

model = sklearn.neighbors.LocalOutlierFactor(novelty=True, contamination=0.39)
model.fit(X_train_sampled_2)

y_pred = model.predict(X_test)

y_pred = (y_pred == (-1 if encoded_values["label1"][1] == "attack" else 1)).astype(int)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

/home/baptiste/cyberml/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(


[[19897  7385]
 [ 7256 10901]]
              precision    recall  f1-score   support

           0       0.73      0.73      0.73     27282
           1       0.60      0.60      0.60     18157

    accuracy                           0.68     45439
   macro avg       0.66      0.66      0.66     45439
weighted avg       0.68      0.68      0.68     45439

